In [64]:
#from urllib.request import urlopen
import spacy
from spacy.cli import link
from spacy.util import get_package_path
from gensim.models import LsiModel

model_name = "de_core_news_sm"
package_path = get_package_path(model_name)
link(model_name, model_name, force=True, model_path=package_path)
nlp = spacy.load("de_core_news_sm")

import operator
import numpy as np
import os.path
from gensim import corpora
from gensim.models import LsiModel
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt
import pandas as pd
import gensim
from gensim import corpora, models
from pprint import pprint


import pyLDAvis.gensim

✔ Linking successful
c:\users\thomas.zoellinger\appdata\local\programs\python\python37\lib\site-packages\de_core_news_sm
-->
c:\users\thomas.zoellinger\appdata\local\programs\python\python37\lib\site-packages\spacy\data\de_core_news_sm
You can now load the model via spacy.load('de_core_news_sm')


ModuleNotFoundError: No module named 'modules'

In [62]:
foo("hash")

TypeError: 'module' object is not callable

In [147]:
A

['Innenstadt',
 'Sachsenhausen',
 'Bahnhofsviertel',
 'Bundesautobahn',
 'Nordend',
 'Gallus',
 'Stadtgebiet',
 'Bockenheim',
 'Bornheim',
 'Ostend',
 'Westend',
 'Niederursel',
 'Griesheim',
 'Höchst',
 'Heddernheim',
 'Rödelheim',
 'Fechenheim',
 'Eschersheim',
 'Dornbusch',
 'Ginnheim',
 'Gutleutviertel',
 'Niederrad',
 'Eckenheim',
 'Praunheim',
 'Schwanheim',
 'Bergen',
 'Preungesheim',
 'Nied',
 'Flughafen',
 'Altstadt',
 'BAB',
 'Nieder',
 'Seckbach',
 'Bahnhofsgebiet',
 'Oberrad',
 'Kalbach',
 'Sossenheim',
 'Unterliederbach',
 'Sindlingen',
 'Bonames',
 'Hausen',
 'Zeilsheim',
 'Riederwald',
 'Autobahn',
 'Gallusviertel',
 'Frankfurter',
 'Riedberg',
 'Bundesstraße',
 'er',
 'Harheim',
 'Nachtrag']

In [26]:
berichte= pd.read_csv(r"../out/Polizeiberichte_raw.csv", sep=";", encoding="UTF-8")
berichte.count()

Timestamp       8073
Ueberschrift    8073
Hauptartikel    8073
Link            8073
dtype: int64

In [128]:
berichte["Ueberschrift"] = berichte["Ueberschrift"].str.replace(r"\d+", "")

In [129]:
def clean_headline(headlines):
    """
    Removes leading and trailing whitespaces from the headline column and removes unnecessary parts.

    :param headlines: A list of headlines
    :return: A list of cleaned headlines
    """
    print("Cleaning headlines...")
    headlines = headlines.apply(lambda x: x.lstrip())
    # Remove beginning from Headline (remove 4digit Number)
    headlines = headlines.str.replace(r"\d{4}", "", regex=True)
    headlines = headlines.str.replace(r"\d{3}", "", regex=True)
    headlines = headlines.str.replace("POL-F:", "")

    headlines = headlines.apply(lambda x: x.lstrip(" -"))
    print("Headlines cleaned.")
    return headlines

In [130]:
def extract_location_from_headline(headlines):
    """
    Extracts the location from the headlines. The location begins after the first occurence of a ":".

    If there are multiple locations, these are separated by a space, "/" or "-", then all secondary locations (those
    after the first separator) will be stored in a new column Location2.
    :param headlines: A list of headlines.
    :return: A list of Locations and a second list of secondary locations
    """
    print("Extracting location from headlines...")
    locations = headlines.apply(lambda x: x[0:x.find(":")])
    locations = locations.str.replace("/", " ")
    locations = locations.str.replace("-", " ")
    locations = locations.str.replace("Frankfurt", "",n=1)
    locations = locations.apply(lambda x: x.lstrip())

    # Extract second location
    secondary_locations = locations.str.split(" ").str[1]

    # Clean location
    locations = locations.str.split(" ").str[0]

    print("Extraction complete.")
    return locations, secondary_locations

In [131]:
berichte["Ueberschrift"] = clean_headline(berichte["Ueberschrift"])

Cleaning headlines...
Headlines cleaned.


In [132]:
berichte["Ort"], berichte["Sekundaerer Ort"] = extract_location_from_headline(berichte["Ueberschrift"])


Extracting location from headlines...
Extraction complete.


In [133]:
berichte

,Timestamp,Ueberschrift,Hauptartikel,Link,Ort,Sekundaerer Ort
0,30.04.2019 – 13:34,Frankfurt-Heddernheim: Inbrandsetzung von Sper...,(ne) Am frühen Samstagmorgen um kurz vor 6:00...,https://www.presseportal.de/blaulicht/pm/4970/...,Heddernheim,NaN
1,30.04.2019 – 13:33,Frankfurt-Bornheim: Zwei Autos abgebrannt,(ka) In der Nacht von Montag auf Dienstag bra...,https://www.presseportal.de/blaulicht/pm/4970/...,Bornheim,NaN
2,30.04.2019 – 08:54,Bundesautobahn : Lastwagenfahrer fährt auf Pol...,(ka) Gestern Abend konnte ein Polizeibeamter ...,https://www.presseportal.de/blaulicht/pm/4970/...,Bundesautobahn,
3,30.04.2019 – 08:40,Frankfurt-Bahnhofsviertel: Bauarbeiter schwer ...,(em) Gestern Nachmittag (29.04.2019) wurde ei...,https://www.presseportal.de/blaulicht/pm/4970/...,Bahnhofsviertel,NaN
4,29.04.2019 – 11:17,Frankfurt-Bornheim: Fahrradcodieraktion zum Fr...,(ne)Das 6. Polizeirevier wird im Rahmen eines...,https://www.presseportal.de/blaulicht/pm/4970/...,Bornheim,NaN
5,29.04.2019 – 11:12,Frankfurt-Flughafen: -Jähriger verletzt Polizi...,(ka) Gestern Nachmittag leistete ein Mann im ...,https://www.presseportal.de/blaulicht/pm/4970/...,Flughafen,NaN
6,28.04.2019 – 12:34,Frankfurt-Nordend: Versuchter Raub in Spielothek,"(fue) Am Sonntag, den 28. April 2019, gegen 0...",https://www.presseportal.de/blaulicht/pm/4970/...,Nordend,NaN
7,26.04.2019 – 14:51,Frankfurt: Verkehrshinweise anlässlich des tr...,(em) Auch dieses Jahr findet das Rad- und Ska...,https://www.presseportal.de/blaulicht/pm/4970/...,,NaN
8,26.04.2019 – 11:54,Frankfurt-Heddernheim: Parfum-Dieb festgenommen,(ne) Der aufmerksame Ladendetektiv eines Eink...,https://www.presseportal.de/blaulicht/pm/4970/...,Heddernheim,NaN
9,26.04.2019 – 11:53,Frankfurt-Nordend: Einbruch in Café,(ne) Ein unbekannter Täter ist gestern Nacht ...,https://www.presseportal.de/blaulicht/pm/4970/...,Nordend,NaN


In [134]:
berichte[berichte['Ort'].str.match("A")]

,Timestamp,Ueberschrift,Hauptartikel,Link,Ort,Sekundaerer Ort
81,08.04.2019 – 14:31,Frankfurt-Altstadt: Motorradfahrer nach Unfall...,(ka) Am Sonntag stürzte ein 54-jähriger Motor...,https://www.presseportal.de/blaulicht/pm/4970/...,Altstadt,NaN
262,24.02.2019 – 12:59,Frankfurt-Altstadt: Taschendieb auf frischer T...,(ka) Zivilbeamte nahmen am Samstagmittag auf ...,https://www.presseportal.de/blaulicht/pm/4970/...,Altstadt,NaN
650,18.11.2018 – 10:24,Frankfurt-Altstadt/Schwanheim: Autodiebe flüch...,(ne)Vier junge Männer sind am frühen Freitagm...,https://www.presseportal.de/blaulicht/pm/4970/...,Altstadt,Schwanheim
681,09.11.2018 – 14:19,Frankfurt-Altstadt: Hinterrücks ausgeraubt,(em) In der Nacht von gestern (08.11.2018) au...,https://www.presseportal.de/blaulicht/pm/4970/...,Altstadt,NaN
841,26.09.2018 – 14:40,Frankfurt-Altstadt: Altstadtfest - Hinweise de...,"(ka) Am Freitag, den 28. September 2018, wird...",https://www.presseportal.de/blaulicht/pm/4970/...,Altstadt,NaN
1015,07.08.2018 – 13:41,Frankfurt-Altstadt: Dealer festgenommen,(ka) Gleich zwei Dealer und einen Käufer nahm...,https://www.presseportal.de/blaulicht/pm/4970/...,Altstadt,NaN
1040,31.07.2018 – 15:32,Frankfurt-Altstadt: Mainfest - Hinweise der F...,"(ka) Am Freitag, den 03. August 2018, wird da...",https://www.presseportal.de/blaulicht/pm/4970/...,Altstadt,NaN
1060,24.07.2018 – 14:33,Frankfurt-A/B: Gemeinsame Pressemitteilung de...,"(em) Wie den Medien bereits bekannt ist, kam ...",https://www.presseportal.de/blaulicht/pm/4970/...,A,B
1071,22.07.2018 – 13:12,Frankfurt-Altstadt/Innenstadt: Sexuelle Beläst...,(em) Gestern Abend (21.07.2018) kam es im Rah...,https://www.presseportal.de/blaulicht/pm/4970/...,Altstadt,Innenstadt
1154,01.07.2018 – 11:39,Frankfurt-Altstadt: Feierlaunige Personengrupp...,(ne)Am späten Samstagabend gegen 23:30 Uhr ka...,https://www.presseportal.de/blaulicht/pm/4970/...,Altstadt,NaN


In [135]:
berichte

,Timestamp,Ueberschrift,Hauptartikel,Link,Ort,Sekundaerer Ort
0,30.04.2019 – 13:34,Frankfurt-Heddernheim: Inbrandsetzung von Sper...,(ne) Am frühen Samstagmorgen um kurz vor 6:00...,https://www.presseportal.de/blaulicht/pm/4970/...,Heddernheim,NaN
1,30.04.2019 – 13:33,Frankfurt-Bornheim: Zwei Autos abgebrannt,(ka) In der Nacht von Montag auf Dienstag bra...,https://www.presseportal.de/blaulicht/pm/4970/...,Bornheim,NaN
2,30.04.2019 – 08:54,Bundesautobahn : Lastwagenfahrer fährt auf Pol...,(ka) Gestern Abend konnte ein Polizeibeamter ...,https://www.presseportal.de/blaulicht/pm/4970/...,Bundesautobahn,
3,30.04.2019 – 08:40,Frankfurt-Bahnhofsviertel: Bauarbeiter schwer ...,(em) Gestern Nachmittag (29.04.2019) wurde ei...,https://www.presseportal.de/blaulicht/pm/4970/...,Bahnhofsviertel,NaN
4,29.04.2019 – 11:17,Frankfurt-Bornheim: Fahrradcodieraktion zum Fr...,(ne)Das 6. Polizeirevier wird im Rahmen eines...,https://www.presseportal.de/blaulicht/pm/4970/...,Bornheim,NaN
5,29.04.2019 – 11:12,Frankfurt-Flughafen: -Jähriger verletzt Polizi...,(ka) Gestern Nachmittag leistete ein Mann im ...,https://www.presseportal.de/blaulicht/pm/4970/...,Flughafen,NaN
6,28.04.2019 – 12:34,Frankfurt-Nordend: Versuchter Raub in Spielothek,"(fue) Am Sonntag, den 28. April 2019, gegen 0...",https://www.presseportal.de/blaulicht/pm/4970/...,Nordend,NaN
7,26.04.2019 – 14:51,Frankfurt: Verkehrshinweise anlässlich des tr...,(em) Auch dieses Jahr findet das Rad- und Ska...,https://www.presseportal.de/blaulicht/pm/4970/...,,NaN
8,26.04.2019 – 11:54,Frankfurt-Heddernheim: Parfum-Dieb festgenommen,(ne) Der aufmerksame Ladendetektiv eines Eink...,https://www.presseportal.de/blaulicht/pm/4970/...,Heddernheim,NaN
9,26.04.2019 – 11:53,Frankfurt-Nordend: Einbruch in Café,(ne) Ein unbekannter Täter ist gestern Nacht ...,https://www.presseportal.de/blaulicht/pm/4970/...,Nordend,NaN


In [136]:
 [e.replace(k, '') for e, k in zip(berichte['Ueberschrift'].astype('str'), berichte['Ort'].astype('str'))]

['Frankfurt-: Inbrandsetzung von Sperrmüll - Polizei sucht Zeugen!',
 'Frankfurt-: Zwei Autos abgebrannt',
 ' : Lastwagenfahrer fährt auf Polizisten zu und versucht zu flüchten',
 'Frankfurt-: Bauarbeiter schwer verletzt',
 'Frankfurt-: Fahrradcodieraktion zum Frühlingsfest',
 'Frankfurt-: -Jähriger verletzt Polizisten',
 'Frankfurt-:  Versuchter Raub in Spielothek',
 'Frankfurt:  Verkehrshinweise anlässlich des traditionellen Radrennens am . Mai',
 'Frankfurt-: Parfum-Dieb festgenommen',
 'Frankfurt-: Einbruch in Café',
 'Frankfurt--Erlenbach: Einbrecher erfolglos geflüchtet',
 'Frankfurt-: Fußgängerin bei Unfall verletzt',
 'Frankfurt-: Geschwindigkeitsmessungen',
 'Frankfurt-: Versuchtes Tötungsdelikt',
 'Frankfurt-: Gute Vorbereitung beim Fahrraddiebstahl',
 'Frankfurt-: Jugendliche knacken Roller und fahren damit',
 'Frankfurt-: Beim Einparken bestohlen',
 'Frankfurt-: Wohnungseinbruch',
 'Frankfurt-: Raub auf Bekleidungsgeschäft',
 'Frankfurt-/Innenstadt: Graffiti gemalt und Drog

In [9]:
berichte= pd.read_csv(r"../out/Polizeiberichte_transformed.csv", sep=";", encoding="UTF-8")

In [10]:
berichte.head()

,Timestamp,Ueberschrift,Hauptartikel,Link,Ort,Sekundaerer Ort,Loc1,Loc2,Loc3,Loc4
0,2019-04-30 13:34:00,heddernheim : inbrandsetzung von sperrmüll ...,(ne) Am frühen Samstagmorgen um kurz vor 6:00...,https://www.presseportal.de/blaulicht/pm/4970/...,heddernheim,NaN,heddernheim,NaN,NaN,NaN
1,2019-04-30 13:33:00,bornheim : zwei autos abgebrannt,(ka) In der Nacht von Montag auf Dienstag bra...,https://www.presseportal.de/blaulicht/pm/4970/...,bornheim,NaN,bornheim,NaN,NaN,NaN
2,2019-04-30 08:54:00,bundesautobahn : lastwagenfahrer fährt auf p...,(ka) Gestern Abend konnte ein Polizeibeamter ...,https://www.presseportal.de/blaulicht/pm/4970/...,bundesautobahn,NaN,bundesautobahn,NaN,NaN,NaN
3,2019-04-30 08:40:00,bahnhofsviertel : bauarbeiter schwer verletzt,(em) Gestern Nachmittag (29.04.2019) wurde ei...,https://www.presseportal.de/blaulicht/pm/4970/...,bahnhofsviertel,NaN,bahnhofsviertel,NaN,NaN,NaN
4,2019-04-29 11:17:00,bornheim : fahrradcodieraktion zum frühlingsfest,(ne)Das 6. Polizeirevier wird im Rahmen eines...,https://www.presseportal.de/blaulicht/pm/4970/...,bornheim,NaN,bornheim,NaN,NaN,NaN


In [25]:
def lemmatize_as_string(document):
    """
    Creates a string of all lemmatized words in the given document, the separator is space.
    :param document: A document to be lemmatized.
    :return: A string of lemmatized words, separated by spaces
    """
    doc = nlp(text=document)
    lemmatized_words = []
    for token in doc:
        lemmatized_words.append(token.lemma_)
    lemmatized_document = " ".join(lemmatized_words)
    return lemmatized_document

lemmatize_as_string("")

''

In [171]:
def get_list_of_Locations(Col):
    Col = Col.str.lower()
    Locations = Col.value_counts()>10
    Locations = list(pd.DataFrame(Col.value_counts()[Locations]).index.values)
    Locations = ' '.join(Locations).split()
    return Locations

In [173]:
A =get_list_of_Locations(berichte["Ort"])

In [162]:
search_list = ('STEEL','IRON','GOLD','SILVER')

df = pd.DataFrame({'a':[123,456,789],'b':['blah blah Steel Gold','blah blah blah','blah blah Gold']})

df.assign(c = df['b'].apply(lambda x: [j for j in x.split() if j.upper() in search_list]))

,a,b,c
0,123,blah blah Steel Gold,"[Steel, Gold]"
1,456,blah blah blah,[]
2,789,blah blah Gold,[Gold]


In [178]:
search_list= A
search_list = tuple(search_list)


In [196]:
berichte.head()

,Timestamp,Ueberschrift,Hauptartikel,Link,Ort,Sekundaerer Ort,Loc_Tuple
0,2019-04-30 13:34:00,Heddernheim : Inbrandsetzung von Sperrmüll ...,(ne) Am frühen Samstagmorgen um kurz vor 6:00...,https://www.presseportal.de/blaulicht/pm/4970/...,Heddernheim,NaN,['Heddernheim']
1,2019-04-30 13:33:00,Bornheim : Zwei Autos abgebrannt,(ka) In der Nacht von Montag auf Dienstag bra...,https://www.presseportal.de/blaulicht/pm/4970/...,Bornheim,NaN,['Bornheim']
2,2019-04-30 08:54:00,Bundesautobahn : Lastwagenfahrer fährt auf P...,(ka) Gestern Abend konnte ein Polizeibeamter ...,https://www.presseportal.de/blaulicht/pm/4970/...,Bundesautobahn,NaN,['Bundesautobahn']
3,2019-04-30 08:40:00,Bahnhofsviertel : Bauarbeiter schwer verletzt,(em) Gestern Nachmittag (29.04.2019) wurde ei...,https://www.presseportal.de/blaulicht/pm/4970/...,Bahnhofsviertel,NaN,['Bahnhofsviertel']
4,2019-04-29 11:17:00,Bornheim : Fahrradcodieraktion zum Frühlingsfest,(ne)Das 6. Polizeirevier wird im Rahmen eines...,https://www.presseportal.de/blaulicht/pm/4970/...,Bornheim,NaN,['Bornheim']


In [208]:
berichte["Test"]= berichte["Loc_Tuple"].str[1:-1].replace("'", "", regex=True).replace(",", "", regex=True)
berichte.head()

,Timestamp,Ueberschrift,Hauptartikel,Link,Ort,Sekundaerer Ort,Loc_Tuple,Test
0,2019-04-30 13:34:00,Heddernheim : Inbrandsetzung von Sperrmüll ...,(ne) Am frühen Samstagmorgen um kurz vor 6:00...,https://www.presseportal.de/blaulicht/pm/4970/...,Heddernheim,NaN,['Heddernheim'],Heddernheim
1,2019-04-30 13:33:00,Bornheim : Zwei Autos abgebrannt,(ka) In der Nacht von Montag auf Dienstag bra...,https://www.presseportal.de/blaulicht/pm/4970/...,Bornheim,NaN,['Bornheim'],Bornheim
2,2019-04-30 08:54:00,Bundesautobahn : Lastwagenfahrer fährt auf P...,(ka) Gestern Abend konnte ein Polizeibeamter ...,https://www.presseportal.de/blaulicht/pm/4970/...,Bundesautobahn,NaN,['Bundesautobahn'],Bundesautobahn
3,2019-04-30 08:40:00,Bahnhofsviertel : Bauarbeiter schwer verletzt,(em) Gestern Nachmittag (29.04.2019) wurde ei...,https://www.presseportal.de/blaulicht/pm/4970/...,Bahnhofsviertel,NaN,['Bahnhofsviertel'],Bahnhofsviertel
4,2019-04-29 11:17:00,Bornheim : Fahrradcodieraktion zum Frühlingsfest,(ne)Das 6. Polizeirevier wird im Rahmen eines...,https://www.presseportal.de/blaulicht/pm/4970/...,Bornheim,NaN,['Bornheim'],Bornheim


In [161]:
berichte.assign(c = berichte['Ueberschrift'].apply(lambda x: [j for j in x.split() if j.lower() in search_list]))

,Timestamp,Ueberschrift,Hauptartikel,Link,Ort,Sekundaerer Ort,LocList,c
0,2019-04-30 13:34:00,Heddernheim : Inbrandsetzung von Sperrmüll ...,(ne) Am frühen Samstagmorgen um kurz vor 6:00...,https://www.presseportal.de/blaulicht/pm/4970/...,Heddernheim,NaN,['Heddernheim'],[Heddernheim]
1,2019-04-30 13:33:00,Bornheim : Zwei Autos abgebrannt,(ka) In der Nacht von Montag auf Dienstag bra...,https://www.presseportal.de/blaulicht/pm/4970/...,Bornheim,NaN,['Bornheim'],[Bornheim]
2,2019-04-30 08:54:00,Bundesautobahn : Lastwagenfahrer fährt auf P...,(ka) Gestern Abend konnte ein Polizeibeamter ...,https://www.presseportal.de/blaulicht/pm/4970/...,Bundesautobahn,NaN,['Bundesautobahn'],[Bundesautobahn]
3,2019-04-30 08:40:00,Bahnhofsviertel : Bauarbeiter schwer verletzt,(em) Gestern Nachmittag (29.04.2019) wurde ei...,https://www.presseportal.de/blaulicht/pm/4970/...,Bahnhofsviertel,NaN,['Bahnhofsviertel'],[Bahnhofsviertel]
4,2019-04-29 11:17:00,Bornheim : Fahrradcodieraktion zum Frühlingsfest,(ne)Das 6. Polizeirevier wird im Rahmen eines...,https://www.presseportal.de/blaulicht/pm/4970/...,Bornheim,NaN,['Bornheim'],[Bornheim]
5,2019-04-29 11:12:00,Flughafen : Jähriger verletzt Polizisten,(ka) Gestern Nachmittag leistete ein Mann im ...,https://www.presseportal.de/blaulicht/pm/4970/...,Flughafen,NaN,['Flughafen'],[Flughafen]
6,2019-04-28 12:34:00,Nordend : Versuchter Raub in Spielothek,"(fue) Am Sonntag, den 28. April 2019, gegen 0...",https://www.presseportal.de/blaulicht/pm/4970/...,Nordend,NaN,['Nordend'],[Nordend]
7,2019-04-26 14:51:00,Frankfurt : Verkehrshinweise anlässlich des ...,(em) Auch dieses Jahr findet das Rad- und Ska...,https://www.presseportal.de/blaulicht/pm/4970/...,NaN,NaN,[],[]
8,2019-04-26 11:54:00,Heddernheim : Parfum Dieb festgenommen,(ne) Der aufmerksame Ladendetektiv eines Eink...,https://www.presseportal.de/blaulicht/pm/4970/...,Heddernheim,NaN,['Heddernheim'],[Heddernheim]
9,2019-04-26 11:53:00,Nordend : Einbruch in Café,(ne) Ein unbekannter Täter ist gestern Nacht ...,https://www.presseportal.de/blaulicht/pm/4970/...,Nordend,NaN,['Nordend'],[Nordend]


In [146]:
search_list = A
pat = r'\b|\b'.join(search_list)
pat2 = r'({})'.format('|'.join(search_list))

df_new= berichte.loc[df.b.str.contains(pat,case=False,na=False)].reset_index(drop=True)
df_new['new_col']=df_new.b.str.upper().str.extract(pat2)
print(df_new)

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match

In [35]:
AAA =['innenstadt', 'sachsenhausen', 'bundesautobahn', 'bahnhofsviertel', 'nordend', 'gallus', 'stadtgebiet', 'bockenheim', 'ostend', 'bornheim', 'westend', 'niederursel', 'griesheim', 'höchst', 'heddernheim', 'rödelheim', 'fechenheim', 'eschersheim', 'dornbusch', 'ginnheim', 'gutleutviertel', 'niederrad', 'eckenheim', 'schwanheim', 'praunheim', 'bergen', 'preungesheim', 'nied', 'flughafen', 'altstadt', 'seckbach', 'bahnhofsgebiet', 'oberrad', 'kalbach', 'nieder-eschbach', 'sossenheim', 'unterliederbach', 'sindlingen', 'hausen', 'bonames', 'zeilsheim', 'riederwald', 'autobahn', 'frankfurter-berg', 'gallusviertel', 'riedberg', 'nieder-erlenbach', 'bundesstraße', 'nachtrag', 'harheim']
AAA = [' ' + x + ' ' for x in AAA]
AAA = ' '.join(AAA).split()
' |'.join(AAA)


'innenstadt |sachsenhausen |bundesautobahn |bahnhofsviertel |nordend |gallus |stadtgebiet |bockenheim |ostend |bornheim |westend |niederursel |griesheim |höchst |heddernheim |rödelheim |fechenheim |eschersheim |dornbusch |ginnheim |gutleutviertel |niederrad |eckenheim |schwanheim |praunheim |bergen |preungesheim |nied |flughafen |altstadt |seckbach |bahnhofsgebiet |oberrad |kalbach |nieder-eschbach |sossenheim |unterliederbach |sindlingen |hausen |bonames |zeilsheim |riederwald |autobahn |frankfurter-berg |gallusviertel |riedberg |nieder-erlenbach |bundesstraße |nachtrag |harheim'

In [36]:
berichte.head()

,Timestamp,Ueberschrift,Hauptartikel,Link
0,30.04.2019 – 13:34,POL-F: 190430 - 479 Frankfurt-Heddernheim: Inb...,(ne) Am frühen Samstagmorgen um kurz vor 6:00...,https://www.presseportal.de/blaulicht/pm/4970/...
1,30.04.2019 – 13:33,POL-F: 190430 - 478 Frankfurt-Bornheim: Zwei A...,(ka) In der Nacht von Montag auf Dienstag bra...,https://www.presseportal.de/blaulicht/pm/4970/...
2,30.04.2019 – 08:54,POL-F: 190430 - 477 Bundesautobahn 5: Lastwage...,(ka) Gestern Abend konnte ein Polizeibeamter ...,https://www.presseportal.de/blaulicht/pm/4970/...
3,30.04.2019 – 08:40,POL-F: 190430 - 476 Frankfurt-Bahnhofsviertel:...,(em) Gestern Nachmittag (29.04.2019) wurde ei...,https://www.presseportal.de/blaulicht/pm/4970/...
4,29.04.2019 – 11:17,POL-F: 190429 - 475 Frankfurt-Bornheim: Fahrra...,(ne)Das 6. Polizeirevier wird im Rahmen eines...,https://www.presseportal.de/blaulicht/pm/4970/...


In [49]:
"""
Utility functions for the cleaning of the scraped data, and transformation, extraction for feature engineering
"""

import pandas as pd
import re
from dateutil.parser import parse

import spacy
from spacy.cli import link
from spacy.util import get_package_path

model_name = "de_core_news_sm"
package_path = get_package_path(model_name)
link(model_name, model_name, force=True, model_path=package_path)
nlp = spacy.load("de_core_news_sm")

# File of Stopwords to be excluded
STOPWORDS_FILE = "Stopwords1.txt"
# File of Topics to be excluded
TopicFilter = "Themenfilter.txt"


def parse_timestamp(timestamps):
    """
    Parses a pandas series of timestamps in the format dd.mm.YYYY - hh:MM into datetime

    :param timestamps: A pandas series of timestamps to be parsed
    :return: The transformed series as a list
    """
    # Remove dash and parse as datetime
    timestamps_parsed = [parse(x.replace("– ", "")) for x in timestamps]
    return timestamps_parsed


def clean_headline(headlines):
    """
    Removes leading and trailing whitespaces from the headline column and removes unnecessary parts.

    :param headlines: A list of headlines
    :return: A list of cleaned headlines
    """
    print("Cleaning headlines...")
    headlines = headlines.apply(lambda x: x.lstrip())
    # Remove beginning from Headline (remove 4digit Number)
    headlines = headlines.str.replace(r"\d{4}", "", regex=True)
    headlines = headlines.str.replace(r"\d{3}", "", regex=True)
    headlines = headlines.str.replace(r"\d+", "")
    headlines = headlines.str.replace("POL-F:", "")
    headlines = headlines.str.replace("Frankfurt-", "")

    headlines = headlines.str.replace("/", " ")
    headlines = headlines.str.replace("-", " ")
    headlines = headlines.str.replace(":", " : ")

    headlines = headlines.str.replace("Frankfurter Berg", "Frankfurter-Berg", n=1)
    headlines = headlines.str.replace("Frankfurter Weihnachtsmarkt", "Frankfurter-Weihnachtsmarkt", n=1)
    headlines = headlines.str.replace("Frankfurter Bahnhofsviertel", "Bahnhofsviertel", n=1)
    headlines = headlines.str.replace("Nieder Erlenbach", "Nieder-Erlenbach", n=1)
    headlines = headlines.str.replace("Nieder Eschbach", "Nieder-Eschbach", n=1)
    headlines = headlines.str.replace("BAB", "Bundesautobahn", n=1)
    headlines = headlines.str.replace("Bad Homburg", "Bad-Homburg", n=1)
    headlines = headlines.str.lower()

    headlines = headlines.apply(lambda x: x.lstrip(" -"))
    print("Headlines cleaned.")
    return headlines


def find_nth_occurrence(string, substring, n):
    """
    Finds the index of the n-th occurence of a given substring in a given string. Counting starts at 0, i.e. the n=0
    gives the index of the first occurrence.
    If the substring is not contained in the string, the result is -1.
    The empty string is part of every string, so if the substring is the empty string, the result will always be n+1.

    :param string: The string that contains occurences of the substring
    :param substring: The substring to search for
    :param n: The desired count of the occurence
    :return: The index of the n-th occurence of the substring in the string
    """
    i = 0
    while n >= 0:
        n -= 1
        i = string.find(substring, i + 1)
    return i


def get_tuple_of_locations(Col):
    """
    Gets a lowercase tuple of the words that appear at least 10 times in the column.
    In this context used to extract locations
    :param Col: Column with the locations
    :return:
    """
    Col = Col.str.lower()
    Locations = Col.value_counts() > 10
    Locations = list(pd.DataFrame(Col.value_counts()[Locations]).index.values)
    Locations = ' '.join(Locations).split()
    Locations = tuple(Locations)
    return Locations


def extract_location_from_headline(headlines):
    """
    Extracts the location from the headlines. The location begins after the first occurence of a ":".

    If there are multiple locations, these are separated by a space, "/" or "-", then all secondary locations (those
    after the first separator) will be stored in a new column Location2.
    :param headlines: A list of headlines.
    :return: A list of Locations and a second list of secondary locations
    """
    print("Extracting location from headlines...")
    locations = headlines.apply(lambda x: x[0:x.find(":")])
    locations = locations.str.replace("/", " ")
    locations = locations.str.replace("-", " ")
    locations = locations.str.replace("frankfurt ", "", n=1)
    locations = locations.str.replace("frankfurter berg", "frankfurter-berg", n=1)
    locations = locations.str.replace("frankfurter weihnachtsmarkt", "frankfurter-weihnachtsmarkt", n=1)
    locations = locations.str.replace("frankfurter bahnhofsviertel", "bahnhofsviertel", n=1)
    locations = locations.str.replace("nieder erlenbach", "nieder-erlenbach", n=1)
    locations = locations.str.replace("nieder eschbach", "nieder-eschbach", n=1)
    locations = locations.str.replace("bab", "bundesautobahn", n=1)
    locations = locations.str.replace("bad homburg", "bad-homburg", n=1)
    locations = locations.str.replace("alt sachsenhausen", "alt-sachsenhausen", n=1)
    locations = locations.apply(lambda x: x.lstrip())

    # Extract second location
    secondary_locations = locations.str.split(" ").str[1]

    # Clean location
    locations = locations.str.split(" ").str[0]

    print("Extraction complete.")
    return locations, secondary_locations


def get_locations_by_tuple(col, tuple):
    """
    Extracts locations from a given column for a given list
    :param col: Column containing the locations
    :param tuple: Tuple with the locations to be extracted
    :return: 3 columns of locations
    """
    col = col.apply(lambda x: [j for j in x.split() if j.lower() in tuple])
    col1 = col.str[0]
    col2 = col.str[1]
    col3 = col.str[2]
    col4 = col.str[3]
    return col1, col2, col3,col4


def remove_word_from_other_column(main_col, words_col1):
    """
    Removes word from a given column provided by a second column
    :param main_col: Column where the words should be removed from
    :param words_col1: Column containing the words that should be removed
    :return: Original column without the words that should be removed
    """

    return [e.replace(k, '') for e, k in zip(main_col.astype('str'), words_col1.astype('str'))]


def extract_description(main_articles):
    """
    Extracts the descriptions from the main article text.

    :param main_articles: A list of article texts
    :return: A list of descriptions
    """
    descriptions = main_articles.apply(lambda x: x[0: find_nth_occurrence(x, ")", 1) + 1])

    # Strip leading and trailing spaces from descriptions
    descriptions = descriptions.apply(lambda x: x.strip())

    return descriptions


def remove_start_from_main_article(main_articles):
    """
    Removes the beginning of the main articles which is not needed.
    :param main_articles: A list of main article texts
    :return: The given list but with the unnecessary beginning cleaned from the texts
    """
    return main_articles.apply(lambda x: x[find_nth_occurrence(x, ")", 1) + 1:])


def extract_place(descriptions):
    """
    Extracts the places from the descriptions
    :param descriptions: A list of descriptions
    :return: The list of places
    """
    places = pd.Series([t.split(" ")[0] for t in descriptions])
    return places


def extract_author(descriptions):
    """
    Extracts the authors from the given list of descriptions
    :param descriptions: A pandas column
    :return: The list of authors
    """
    # Get the 10 first characters
    authors = descriptions.str[:10]
    # Find ")" and get the text before
    authors = authors.apply(lambda x: x[:find_nth_occurrence(x, ")", 0) + 1])
    # Removing everything non-alphanumeric
    authors = [re.sub("[^a-zA-Z äöüÄÖÜß]+", "", x) for x in authors]
    # Convert to Pandas Series again
    authors = pd.Series(authors)
    # Strip white spaces or return empty string and convert to lower
    authors = authors.apply(lambda x: x.lstrip() if pd.notnull(x) else "").str.lower()
    return authors


def lemmatize_as_string(document):
    """
    Creates a string of all lemmatized words in the given document, the separator is space.
    :param document: A document to be lemmatized.
    :return: A string of lemmatized words, separated by spaces
    """
    try:
        doc = nlp(text=document)
        lemmatized_words = []
        for token in doc:
            lemmatized_words.append(token.lemma_)
        lemmatized_document = " ".join(lemmatized_words)
        return lemmatized_document
    except:
        return ""


def get_stopword_list():
    """
    Gets the stopwords from the file.
    :return: List of stopwords
    """
    stopwords = open(r"../resources/%s" % STOPWORDS_FILE, "r")
    stopwords = list(stopwords)
    stopwords = list(map(lambda s: s.strip(), stopwords))
    return stopwords


def get_topicfilter_list():
    """
    Gets the words that should not be included in topics from the file.
    :return: List of words/topics
    """
    Themenfilter = open(r"../resources/%s" % TopicFilter, "r")
    Themenfilter = list(Themenfilter)
    Themenfilter = list(map(lambda s: s.strip(), Themenfilter))
    return Themenfilter


def FilterTopics(DF, Column_Name, Filterwords_List):
    """
    Keeps only the rows of a Dataframe where the filter words are not present in the specified column
    :param DF: DataFrame which should be filtered
    :param Column_Name: Name of the Column of the Dataframe which should be filtered. Must be given as a string
    :param Filterwords_List: List of topics/words that should be filtered
    :return:
    """
    return DF[~DF[Column_Name].str.lower().str.contains('|'.join(Filterwords_List), na=False)]


def get_string_from_list(liste):
    """
    Turns the given list into a string, with the list elements separated by spaces.
    :param liste: A list of elements that can be turned into strings
    :return: A string of the list elements separated by spaces
    """
    return ' '.join([str(x) for x in liste])


def remove_stopwords(document):
    """
    Removes all stopwords from document.
    :param document: A document containing words.
    :return: The document without the stopwords
    """
    stopwords = get_stopword_list()
    words = document.split(" ")
    words_without_stopwords = [x for x in words if x not in stopwords]
    document_no_stopwords = get_string_from_list(words_without_stopwords)
    return document_no_stopwords


def clean_column(documents):
    """
    Clean the whole column: remove white spaces, digits and transform to lowercase.
    :param documents: A list of documents to clean
    :return: A list of cleaned documents
    """
    # Removing everything non-alphanumeric
    print("clean######################")
    documents = pd.Series(documents)
    print((type(documents)))
    documents.fillna(value="AAAAAAAAAAAAAAA", inplace=True)
    documents = [re.sub("[^a-zA-Z äöüÄÖÜß]+", " ", x) for x in documents]
    # Converting to lowercase
    documents = [x.lower() for x in documents]
    documents = pd.Series(documents)
    # Remove digits
    documents = documents.str.replace(r'\d+', '')
    print("--------------------------------------------------------------")
    print("--------------------------------------------------------------")
    print(len(documents))

    return documents


def split_compound_words(documents):
    """
    Split German compound words in all documents into their parts
    :param documents: A list of documents
    :return: A list of strings, where each string consists of the words from the input with compound words split.
    Word separator in the strings is a space.
    """
    # Create a matrix of the words from the documents. The n-th col contains the n-th word in the document.
    words = documents.str.split(' ', expand=True)
    words = words.rename(columns=lambda x: "string_{}".format(x + 1))

    for word_position in words.columns:
        documents = words[word_position]
        documents.fillna(value="", inplace=True)

        # Splitting
        compound_words_split = documents.apply(char_split.split_compound)
        compound_words_split = [item[0] for item in compound_words_split]
        compound_words_split = pd.DataFrame(compound_words_split)
        compound_words_split.columns = ["confidence", "first_word", "second_word"]

        # For non-compound words first_word and second_word are equal. To evade doubling these words, remove second word
        compound_words_split.loc[
            compound_words_split["first_word"] == compound_words_split["second_word"], "second_word"] = ""

        # Only keep words that received positive confidence from the splitter
        words_to_drop = compound_words_split["confidence"] < 0
        compound_words_split[words_to_drop] = " "
        compound_words_split["combined"] = compound_words_split["first_word"] + " " + compound_words_split[
            "second_word"]
        words[word_position] = compound_words_split["combined"]
        words.fillna(value="", inplace=True)
    # Connect the words for each document into a long, space-separated string
    words["all_combined"] = words.apply(' '.join, axis=1)
    return words["all_combined"]


def lemmatize_document_list(documents):
    """
    Lemmatizes the words in all given documents.
    :param documents: A list of documents made up of words to be lemmatized.
    :return: A list where each item is a string of the lemmatized words for each document, separated by spaces.
    """
    # Lemmatize
    lemmatized_berichte = [lemmatize_as_string(x) for x in documents]
    # Make everything lowercase
    lemmatized_berichte = [x.lower() for x in lemmatized_berichte]
    # Concatenate as a long string
    lemmatized_berichte = [' '.join(x.split(" ")) for x in lemmatized_berichte]
    return lemmatized_berichte


✔ Linking successful
c:\users\thomas.zoellinger\appdata\local\programs\python\python37\lib\site-packages\de_core_news_sm
-->
c:\users\thomas.zoellinger\appdata\local\programs\python\python37\lib\site-packages\spacy\data\de_core_news_sm
You can now load the model via spacy.load('de_core_news_sm')


In [50]:
berichte = pd.read_csv(r"../out/Polizeiberichte_raw.csv",
                       sep=";",
                       encoding="UTF-8")

# Parse timestamp strings as datetime

# Clean headlines
berichte["Ueberschrift"] = clean_headline(berichte["Ueberschrift"])

# Drop rows that are just a supplement or a correction
berichte = berichte[~berichte["Ueberschrift"].str.contains("nachtrag", na=False)]
berichte = berichte[~berichte["Ueberschrift"].str.contains("korrektur", na=False)]

# Drop rows with NANs
berichte = berichte.dropna()

# Extract locations
berichte["Ort"], berichte["Sekundaerer Ort"] = extract_location_from_headline(berichte["Ueberschrift"])

Loc_Tuple = get_tuple_of_locations(berichte["Ort"])

#berichte["LocList"] = berichte["Ueberschrift"].apply(lambda x: [i for i in Loc_List if i in x])

berichte["Loc1"], berichte["Loc2"], berichte["Loc3"], berichte["Loc4"] = get_locations_by_tuple(berichte["Ueberschrift"], Loc_Tuple)

print(list(Loc_Tuple))
# Remove location from headline
berichte["Ueberschrift"] = berichte["Ueberschrift"].str.replace(' |'.join(list(Loc_Tuple)), '')





# Extract place and author from description
berichte["Author"] = extract_author(berichte["Hauptartikel"])

# Remove all dates from headline
berichte["Ueberschrift"] = berichte["Ueberschrift"].str.replace(r'\d+', '')
# Clean headline
berichte["Ueberschrift_clean"] = clean_column(berichte["Ueberschrift"])

# Lemmatize words in headline text
berichte["Ueberschrift_lem"] = lemmatize_document_list(berichte["Ueberschrift_clean"])
# Remove stopwords from headline
berichte["Ueberschrift_lem_no_stop"] = berichte["Ueberschrift_lem"].apply(remove_stopwords)

# Split compound words
berichte["Ueberschrift_split"] = split_compound_words(berichte["Ueberschrift_lem_no_stop"])
# Clean headline
berichte["Ueberschrift_clean2"] = clean_column(berichte["Ueberschrift_split"])
# Lemmatize words in headline text
berichte["Ueberschrift_lem2"] = lemmatize_document_list(berichte["Ueberschrift_clean2"])
# Clean headline again after lemmatization
berichte["Ueberschrift_lem_clean2"] = clean_column(berichte["Ueberschrift_lem2"])
# Remove stopwords from headline

print("--------------------------------------------------------------")
print("--------------------------------------------------------------")

#berichte["Ueberschrift_lem_clean2"].fillna(value="", inplace=True)
print(len(berichte["Ueberschrift_lem_clean2"]))
print(berichte["Ueberschrift_lem_clean2"].isna().sum())
print(clean_column(berichte["Ueberschrift_lem2"].isna().sum()))


berichte["Ueberschrift_lem_clean_no_stop"] = berichte["Ueberschrift_lem_clean2"].apply(remove_stopwords)

Cleaning headlines...
Headlines cleaned.
Extracting location from headlines...
Extraction complete.
['innenstadt', 'sachsenhausen', 'bundesautobahn', 'bahnhofsviertel', 'nordend', 'gallus', 'stadtgebiet', 'ostend', 'bornheim', 'bockenheim', 'westend', 'niederursel', 'griesheim', 'höchst', 'heddernheim', 'fechenheim', 'eschersheim', 'rödelheim', 'dornbusch', 'ginnheim', 'gutleutviertel', 'eckenheim', 'niederrad', 'schwanheim', 'praunheim', 'bergen', 'preungesheim', 'nied', 'altstadt', 'flughafen', 'oberrad', 'seckbach', 'bahnhofsgebiet', 'kalbach', 'nieder-eschbach', 'sossenheim', 'sindlingen', 'unterliederbach', 'hausen', 'bonames', 'zeilsheim', 'riederwald', 'frankfurter-berg', 'autobahn', 'gallusviertel', 'riedberg', 'bundesstraße', 'nieder-erlenbach', 'harheim']
clean######################
<class 'pandas.core.series.Series'>
--------------------------------------------------------------
--------------------------------------------------------------
7835


NameError: name 'char_split' is not defined